# 201600282 엄기산

## 1. Motivation
 - 영화관에 방문하면 한번쯤은 혼자 예매를 하여 상영관에 혼자만 있는 경우가 있다. 이때는 정말 주변 사람들에게 방해를 받지않아 몰입이 쉽게 된다. 반대로 주변 사람들에 의해 영화에 집중을 못하는 경우가 종종 있다. 그로 인하여 좀더 사람이 영화관에 덜 방문하는 날짜를 확인하고 싶었다.
  
 - 그렇다면 예매 인원에 가장 영향력이 있을만한 요소를 확인을 해보았다.
 
 - 먼저 흥행영화가 떠올랐지만 그 경우에는 나 또한 사람이 많든 적든 영화관을 방문하기 때문에 제외하였다.
 
 - 그 다음으로 개봉 영화의 장르가 떠올랐다. 하지만 이 또한 개인의 취향이기 때문에 제외하였다.
 
 - 그 다음은 개봉 영화들의 광고비를 생각해 보았지만 데이터를 찾기 어려움이 있어 제외하였다.
 
 - 마지막으로 날씨를 생각해보고 좋은날과 안좋은날에 대해 영화 예매인원의 관계를 파악하고자 한다. 날씨 데이터는 충분히 공공데이터로 공개가 되어있으니 데이터 접근성이 좋았다.
 
 
 
 
## 2.데이터 수집 과정

 - 구체적인 계획
   1. 먼저 영화 일별 예매 인원수 데이터를 받아야함
    -> 웹 크롤링 사용
    
   2. 일별 서울지역 날씨 데이터 받기 
    -> 기상자료개방포털은 오직 기온과 습도 바람 데이터뿐 
    -> 날씨가 좋은지 안좋은지 대략적인 추측만 가능하여 정확한 데이터 마이닝이 안될것 같음
    -> 네이버 날씨에서는 과거 날씨데이터 사진으로 가지고있음 -> 하지만 robots.txt에서 메인페이지를 제외한 모든 페이지 크롤링 금지한다 함.
    -> 다른 웹페이지를 탐색 
    -> 웨더아이
    
   3. 역대 주말
    -> 빨간날 모두 불러오는건 어찌할 것인가?
    
## 3. 데이터 전처리 과정
 Map Reduce는 어찌 사용할 것인가???
    
   3. 좋은날과 안좋은날의 구분
    -> 웨더아이에서 데이터 받아와서 몇개의 기후가 있는지 파악
    -> 모든 날씨에 대하여 점수로 평가함
   
   4. 주말과 평일의 구분
    -> 주말에는 언제나 몰리기 때문에 주말과 평일을 구분하지 않으면 정확한 데이터 마이닝이 안될거 같음
    -> 주말과 평일을 따로 나눠서 분석을 하여 나중에 비슷한지 파악하기
    
   5. 역대 흥행영화의 구분
    -> 언제나 흥행영화는 존재
    -> 흥행영화가 있을 때의 데이터를 제외하게 되면 대부분의 데이터가 사라질것이라 판단.
    -> 대신 분석하는 기간을 10년으로 잡아 충분하게 많은 데이터 파악해서 어느정도 흥행영화의 영향을 감소시킴
   
   6. COVID19
    -> 코로나로 인한 영향을 제외하기 위해 17년도 까지의 데이터만 받는다.
    
   7. 영화별 예매 인원
    -> 영화별 예매 인원 데이터가 존재하니 일별 전체 예매인원을 SUM() 함수를 통해서 파악한다.
    
## 4. 데이터 결과 분석
   8. 현재 임의로 부여한것 분석
    -> 날씨에 대한 점수 평가 기준이 주관적임
    -> 평가 기준을 계속 다르게 해보면서 분석을 해야한다.
   
   9. 주말과 평일의 구분
    -> 주말과 평일에 대하여 날씨의 영향이 얼마나 큰지 분석한다.

2. Model
 Map Reduce가 적합하다고 생각됨
 

In [2]:
import pandas as pd
import csv
import numpy as np

# 1)크롤링 데이터 확인하기

In [3]:
df_theater = pd.DataFrame(pd.read_csv("./theater.csv"))
df_theater

,날짜,총 예매인원
0,2008-01-01,194337
1,2008-01-02,67939
2,2008-01-03,71980
3,2008-01-04,76695
4,2008-01-05,168028
...,...,...
3648,2017-12-27,248380
3649,2017-12-28,192597
3650,2017-12-29,259585
3651,2017-12-30,391572


In [4]:
df_weather = pd.DataFrame(pd.read_csv("./weather.csv"))
df_weather

,날짜,날씨
0,2008-01-01,맑음
1,2008-01-02,맑음
2,2008-01-03,구름조금
3,2008-01-04,구름조금
4,2008-01-05,맑음
...,...,...
3648,2017-12-27,맑음
3649,2017-12-28,흐림/눈/안개
3650,2017-12-29,구름조금/눈/안개
3651,2017-12-30,흐림/눈/비/안개/진눈개비


# 2) 데이터 전처리
## 2-1) 날씨데이터 전처리

날씨를 구분하기 위해서 총 몇개의 날씨 종류가 있는지 확인을 한다.

In [5]:
we_list = df_weather["날씨"].values
we_set = set(we_list)
we_value = list(we_set)
weather_list = []
for value in we_value:
    temp = value.split('/')
    for val in temp:
        weather_list.append(val)
weather_set = set(weather_list)
weather_value = list(weather_set)
weather_value

['눈',
 '소낙눈',
 '황사',
 '구름조금',
 '비',
 '이슬비',
 '구름많음',
 '천둥번개',
 '진눈개비',
 '소나기',
 '우박',
 '안개',
 '맑음',
 '흐림']

즉 총 14개의 날씨로 구분이 가능하며 이 날씨들은 하루에 여러개가 중복되에 나타날 수 있다.

그러면 이제 frame_we에 있는 날씨 열을 14개의 열로 쪼개어 나타내보자

그 후 각 날씨에 따라 영화 예매에 얼마나 미치는지 확인을 한다.

In [6]:
df_weather['맑음']=0
df_weather['비']=0
df_weather['눈']=0
df_weather['진눈개비']=0
df_weather['황사']=0
df_weather['흐림']=0
df_weather['구름많음']=0
df_weather['구름조금']=0
df_weather['안개']=0
df_weather['소낙눈']=0
df_weather['소나기']=0
df_weather['이슬비']=0
df_weather['천둥번개']=0
df_weather['우박']=0
df_weather

for index, data in enumerate(df_weather['날씨']):
    value = data.split('/')
    for val in value:
        df_weather.loc[index,val] = 1
del df_weather['날씨']

In [7]:
df_weather

,날짜,맑음,비,눈,진눈개비,황사,흐림,구름많음,구름조금,안개,소낙눈,소나기,이슬비,천둥번개,우박
0,2008-01-01,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2008-01-02,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2008-01-03,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,2008-01-04,0,0,0,0,0,0,0,1,0,0,0,0,0,0
4,2008-01-05,1,0,0,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2017-12-27,1,0,0,0,0,0,0,0,0,0,0,0,0,0
3649,2017-12-28,0,0,1,0,0,1,0,0,1,0,0,0,0,0
3650,2017-12-29,0,0,1,0,0,0,0,1,1,0,0,0,0,0
3651,2017-12-30,0,1,1,1,0,1,0,0,1,0,0,0,0,0


## 2-2) DataFrame 합친 후 요일별로 나누기

### df_weather = 요일별 나누기 전 데이터

In [8]:
df_weather['예매인원'] = df_theater['총 예매인원']


num_of_day ={
    0 : '월',
    1 : '화',
    2 : '수',
    3 : '목',
    4 : '금',
    5 : '토',
    6 : '일'
}

df_weather['날짜'] = pd.to_datetime(df_weather['날짜'],format='%Y-%m-%d')

df_weather['요일'] = [num_of_day[k] for k in df_weather['날짜'].dt.dayofweek]

df_weather

,날짜,맑음,비,눈,진눈개비,황사,흐림,구름많음,구름조금,안개,소낙눈,소나기,이슬비,천둥번개,우박,예매인원,요일
0,2008-01-01,1,0,0,0,0,0,0,0,0,0,0,0,0,0,194337,화
1,2008-01-02,1,0,0,0,0,0,0,0,0,0,0,0,0,0,67939,수
2,2008-01-03,0,0,0,0,0,0,0,1,0,0,0,0,0,0,71980,목
3,2008-01-04,0,0,0,0,0,0,0,1,0,0,0,0,0,0,76695,금
4,2008-01-05,1,0,0,0,0,0,0,0,0,0,0,0,0,0,168028,토
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2017-12-27,1,0,0,0,0,0,0,0,0,0,0,0,0,0,248380,수
3649,2017-12-28,0,0,1,0,0,1,0,0,1,0,0,0,0,0,192597,목
3650,2017-12-29,0,0,1,0,0,0,0,1,1,0,0,0,0,0,259585,금
3651,2017-12-30,0,1,1,1,0,1,0,0,1,0,0,0,0,0,391572,토


### df_weather 데이터 요일별로 나누기 df_요일

In [9]:
df_mon = pd.DataFrame(df_weather.iloc[6]).T
for i in range(1,(3652//7)):
    idx = i*7 + 6
    df_mon = df_mon.append(df_weather.iloc[idx])    
df_mon = df_mon.drop(['요일'],axis=1)



df_tue = pd.DataFrame(df_weather.iloc[0]).T
for i in range(1,(3652//7)+1):
    idx = i*7 + 0
    df_tue = df_tue.append(df_weather.iloc[idx])    
df_tue = df_tue.drop(['요일'],axis=1)



df_wed = pd.DataFrame(df_weather.iloc[1]).T
for i in range(1,(3652//7)+1):
    idx = i*7 + 1
    df_wed = df_wed.append(df_weather.iloc[idx])    
df_wed = df_wed.drop(['요일'],axis=1)



df_thr = pd.DataFrame(df_weather.iloc[2]).T
for i in range(1,(3652//7)+1):
    idx = i*7 + 2
    df_thr = df_thr.append(df_weather.iloc[idx])   
df_thr = df_thr.drop(['요일'],axis=1)



df_fri = pd.DataFrame(df_weather.iloc[3]).T
for i in range(1,(3652//7)+1):
    idx = i*7 + 3
    df_fri = df_fri.append(df_weather.iloc[idx])    

df_fri = df_fri.drop(['요일'],axis=1)



df_sat = pd.DataFrame(df_weather.iloc[4]).T
for i in range(1,(3652//7)+1):
    idx = i*7 + 4
    df_sat = df_sat.append(df_weather.iloc[idx])    

df_sat = df_sat.drop(['요일'],axis=1)



df_sun = pd.DataFrame(df_weather.iloc[5]).T
for i in range(1,(3652//7)+1):
    idx = i*7 + 5
    df_sun = df_sun.append(df_weather.iloc[idx])    

df_sun = df_sun.drop(['요일'],axis=1)

In [10]:
df_mon

,날짜,맑음,비,눈,진눈개비,황사,흐림,구름많음,구름조금,안개,소낙눈,소나기,이슬비,천둥번개,우박,예매인원
6,2008-01-07,0,0,0,0,0,0,1,0,1,0,0,0,0,0,54651
13,2008-01-14,1,0,0,0,0,0,0,0,0,0,0,0,0,0,80377
20,2008-01-21,0,0,1,0,0,1,0,0,0,0,0,0,0,0,85027
27,2008-01-28,0,0,0,0,0,1,0,0,0,0,0,0,0,0,67832
34,2008-02-04,1,0,0,0,0,0,0,0,1,0,0,0,0,0,72618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3618,2017-11-27,1,0,0,0,0,0,0,0,0,0,0,0,0,0,65680
3625,2017-12-04,1,1,0,0,0,0,0,0,1,0,0,0,0,0,65087
3632,2017-12-11,1,0,0,0,0,0,0,0,0,0,0,0,0,0,61330
3639,2017-12-18,0,0,1,0,0,0,1,0,1,0,0,0,0,0,96302


## 2-3) 요일에 대한 가중치 값 구해서 요일이 영화 예매에 미치는 영향 줄이기

### df = 요일에 대한 가중치값 적용한 데이터

In [12]:
date = pd.DataFrame([['월',df_mon['예매인원'].mean()],
                     ['화',df_tue['예매인원'].mean()],
                     ['수',df_wed['예매인원'].mean()],
                     ['목',df_thr['예매인원'].mean()],
                     ['금',df_fri['예매인원'].mean()],
                     ['토',df_sat['예매인원'].mean()],
                     ['일',df_sun['예매인원'].mean()]],columns=['요일','평균값'])

weight = 150000/date['평균값']
weight

0    1.617486
1    1.676706
2    1.453708
3    1.343242
4    1.133228
5    0.627591
6    0.713524
Name: 평균값, dtype: float64

### 가중치 적용후 다시 데이터 합치기 
### 합친 결과 = df

In [13]:
df_mon['예매인원'] = df_mon['예매인원']*weight[0]

df_tue['예매인원'] = df_tue['예매인원']*weight[1]

df_wed['예매인원'] = df_wed['예매인원']*weight[2]

df_thr['예매인원'] = df_thr['예매인원']*weight[3]

df_fri['예매인원'] = df_fri['예매인원']*weight[4]

df_sat['예매인원'] = df_sat['예매인원']*weight[5]

df_sun['예매인원'] = df_sun['예매인원']*weight[6]

df = pd.concat([df_mon,df_tue,df_wed,df_thr,df_fri,df_sat,df_sun])
df = df.sort_values(by=['날짜'])

df

,날짜,맑음,비,눈,진눈개비,황사,흐림,구름많음,구름조금,안개,소낙눈,소나기,이슬비,천둥번개,우박,예매인원
0,2008-01-01,1,0,0,0,0,0,0,0,0,0,0,0,0,0,325846.102374
1,2008-01-02,1,0,0,0,0,0,0,0,0,0,0,0,0,0,98763.490177
2,2008-01-03,0,0,0,0,0,0,0,1,0,0,0,0,0,0,96686.535035
3,2008-01-04,0,0,0,0,0,0,0,1,0,0,0,0,0,0,86912.907665
4,2008-01-05,1,0,0,0,0,0,0,0,0,0,0,0,0,0,105452.78482
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2017-12-27,1,0,0,0,0,0,0,0,0,0,0,0,0,0,361072.074803
3649,2017-12-28,0,0,1,0,0,1,0,0,1,0,0,0,0,0,258704.314923
3650,2017-12-29,0,0,1,0,0,0,0,1,1,0,0,0,0,0,294168.943688
3651,2017-12-30,0,1,1,1,0,1,0,0,1,0,0,0,0,0,245746.886576


### 예매인원의 날씨 변화에 대한 변화량 계산

In [14]:
for i in range(1,3653):
    df['예매인원'][i] -= df['예매인원'][i-1]
    
df = df.drop([df.index[0]])
df

<ipython-input-14-e62c03b88b82>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['예매인원'][i] -= df['예매인원'][i-1]


,날짜,맑음,비,눈,진눈개비,황사,흐림,구름많음,구름조금,안개,소낙눈,소나기,이슬비,천둥번개,우박,예매인원
1,2008-01-02,1,0,0,0,0,0,0,0,0,0,0,0,0,0,-227082.612197
2,2008-01-03,0,0,0,0,0,0,0,1,0,0,0,0,0,0,323769.147232
3,2008-01-04,0,0,0,0,0,0,0,1,0,0,0,0,0,0,-236856.239568
4,2008-01-05,1,0,0,0,0,0,0,0,0,0,0,0,0,0,342309.024388
5,2008-01-06,0,0,0,0,0,1,0,0,1,0,0,0,0,0,-235789.879978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3648,2017-12-27,1,0,0,0,0,0,0,0,0,0,0,0,0,0,486527.541687
3649,2017-12-28,0,0,1,0,0,1,0,0,1,0,0,0,0,0,-227823.226764
3650,2017-12-29,0,0,1,0,0,0,0,1,1,0,0,0,0,0,521992.170452
3651,2017-12-30,0,1,1,1,0,1,0,0,1,0,0,0,0,0,-276245.283876


In [15]:
index = ['기준 날씨','날씨 변화','평균 변화율','데이터 갯수']

diff = sum(df['예매인원'])//len(df['예매인원'])

In [16]:
diff

75010.0

### 날씨 변화량에 따른 예매 변화율 계산 ( 다변수 함수의 편미분값 )

In [17]:
data1_0 = []
data1_1 = []
data1_2 = []
data1_3 = []


for i in range(2,3653):
    if df['맑음'][i-1]==0 and df['맑음'][i] ==0:
        data1_0.append(df['예매인원'][i])
    elif df['맑음'][i-1]==0 and df['맑음'][i] ==1:
        data1_1.append(df['예매인원'][i])
    elif df['맑음'][i-1]==1 and df['맑음'][i] ==0:
        data1_2.append(df['예매인원'][i])
    elif df['맑음'][i-1]==1 and df['맑음'][i] ==1:
        data1_3.append(df['예매인원'][i])

df_data1 = pd.DataFrame([['맑음',0,(sum(data1_0)/len(data1_0)-diff)/diff,len(data1_0)],
                         ['맑음',1,(sum(data1_1)/len(data1_1)-diff)/diff,len(data1_1)],
                         ['맑음',2,(sum(data1_2)/len(data1_2)-diff)/diff,len(data1_2)],
                         ['맑음',3,(sum(data1_3)/len(data1_3)-diff)/diff,len(data1_3)]],columns = index)




data2_0 = []
data2_1 = []
data2_2 = []
data2_3 = []


for i in range(2,3653):
    if df['비'][i-1]==0 and df['비'][i] ==0:
        data2_0.append(df['예매인원'][i])
    elif df['비'][i-1]==0 and df['비'][i] ==1:
        data2_1.append(df['예매인원'][i])
    elif df['비'][i-1]==1 and df['비'][i] ==0:
        data2_2.append(df['예매인원'][i])
    elif df['비'][i-1]==1 and df['비'][i] ==1:
        data2_3.append(df['예매인원'][i])

df_data2 = pd.DataFrame([['비',0,(sum(data2_0)/len(data2_0)-diff)/diff,len(data2_0)],
                         ['비',1,(sum(data2_1)/len(data2_1)-diff)/diff,len(data2_1)],
                         ['비',2,(sum(data2_2)/len(data2_2)-diff)/diff,len(data2_2)],
                         ['비',3,(sum(data2_3)/len(data2_3)-diff)/diff,len(data2_3)]],columns = index)




data3_0 = []
data3_1 = []
data3_2 = []
data3_3 = []


for i in range(2,3653):
    if df['눈'][i-1]==0 and df['눈'][i] ==0:
        data3_0.append(df['예매인원'][i])
    elif df['눈'][i-1]==0 and df['눈'][i] ==1:
        data3_1.append(df['예매인원'][i])
    elif df['눈'][i-1]==1 and df['눈'][i] ==0:
        data3_2.append(df['예매인원'][i])
    elif df['눈'][i-1]==1 and df['눈'][i] ==1:
        data3_3.append(df['예매인원'][i])

df_data3 = pd.DataFrame([['눈',0,(sum(data3_0)/len(data3_0)-diff)/diff,len(data3_0)],
                         ['눈',1,(sum(data3_1)/len(data3_1)-diff)/diff,len(data3_1)],
                         ['눈',2,(sum(data3_2)/len(data3_2)-diff)/diff,len(data3_2)],
                         ['눈',3,(sum(data3_3)/len(data3_3)-diff)/diff,len(data3_3)]],columns = index)




data4_0 = []
data4_1 = []
data4_2 = []
data4_3 = []


for i in range(2,3653):
    if df['진눈개비'][i-1]==0 and df['진눈개비'][i] ==0:
        data4_0.append(df['예매인원'][i])
    elif df['진눈개비'][i-1]==0 and df['진눈개비'][i] ==1:
        data4_1.append(df['예매인원'][i])
    elif df['진눈개비'][i-1]==1 and df['진눈개비'][i] ==0:
        data4_2.append(df['예매인원'][i])
    elif df['진눈개비'][i-1]==1 and df['진눈개비'][i] ==1:
        data4_3.append(df['예매인원'][i])

df_data4 = pd.DataFrame([['진눈개비',0,(sum(data4_0)/len(data4_0)-diff)/diff,len(data4_0)],
                         ['진눈개비',1,(sum(data4_1)/len(data4_1)-diff)/diff,len(data4_1)],
                         ['진눈개비',2,(sum(data4_2)/len(data4_2)-diff)/diff,len(data4_2)],
                         ['진눈개비',3,(sum(data4_3)/len(data4_3)-diff)/diff,len(data4_3)]],columns = index)




data5_0 = []
data5_1 = []
data5_2 = []
data5_3 = []


for i in range(2,3653):
    if df['황사'][i-1]==0 and df['황사'][i] ==0:
        data5_0.append(df['예매인원'][i])
    elif df['황사'][i-1]==0 and df['황사'][i] ==1:
        data5_1.append(df['예매인원'][i])
    elif df['황사'][i-1]==1 and df['황사'][i] ==0:
        data5_2.append(df['예매인원'][i])
    elif df['황사'][i-1]==1 and df['황사'][i] ==1:
        data5_3.append(df['예매인원'][i])

df_data5 = pd.DataFrame([['황사',0,sum(data5_0)/len(data5_0)/diff,len(data5_0)],
                         ['황사',1,sum(data5_1)/len(data5_1)/diff,len(data5_1)],
                         ['황사',2,sum(data5_2)/len(data5_2)/diff,len(data5_2)],
                         ['황사',3,sum(data5_3)/len(data5_3)/diff,len(data5_3)]],columns = index)




data6_0 = []
data6_1 = []
data6_2 = []
data6_3 = []


for i in range(2,3653):
    if df['흐림'][i-1]==0 and df['흐림'][i] ==0:
        data6_0.append(df['예매인원'][i])
    elif df['흐림'][i-1]==0 and df['흐림'][i] ==1:
        data6_1.append(df['예매인원'][i])
    elif df['흐림'][i-1]==1 and df['흐림'][i] ==0:
        data6_2.append(df['예매인원'][i])
    elif df['흐림'][i-1]==1 and df['흐림'][i] ==1:
        data6_3.append(df['예매인원'][i])

df_data6 = pd.DataFrame([['흐림',0,sum(data6_0)/len(data6_0)/diff,len(data6_0)],
                         ['흐림',1,sum(data6_1)/len(data6_1)/diff,len(data6_1)],
                         ['흐림',2,sum(data6_2)/len(data6_2)/diff,len(data6_2)],
                         ['흐림',3,sum(data6_3)/len(data6_3)/diff,len(data6_3)]],columns = index)




data7_0 = []
data7_1 = []
data7_2 = []
data7_3 = []


for i in range(2,3653):
    if df['구름많음'][i-1]==0 and df['구름많음'][i] ==0:
        data7_0.append(df['예매인원'][i])
    elif df['구름많음'][i-1]==0 and df['구름많음'][i] ==1:
        data7_1.append(df['예매인원'][i])
    elif df['구름많음'][i-1]==1 and df['구름많음'][i] ==0:
        data7_2.append(df['예매인원'][i])
    elif df['구름많음'][i-1]==1 and df['구름많음'][i] ==1:
        data7_3.append(df['예매인원'][i])

df_data7 = pd.DataFrame([['구름많음',0,sum(data7_0)/len(data7_0)/diff,len(data7_0)],
                         ['구름많음',1,sum(data7_1)/len(data7_1)/diff,len(data7_1)],
                         ['구름많음',2,sum(data7_2)/len(data7_2)/diff,len(data7_2)],
                         ['구름많음',3,sum(data7_3)/len(data7_3)/diff,len(data7_3)]],columns = index)




data8_0 = []
data8_1 = []
data8_2 = []
data8_3 = []


for i in range(2,3653):
    if df['구름조금'][i-1]==0 and df['구름조금'][i] ==0:
        data8_0.append(df['예매인원'][i])
    elif df['구름조금'][i-1]==0 and df['구름조금'][i] ==1:
        data8_1.append(df['예매인원'][i])
    elif df['구름조금'][i-1]==1 and df['구름조금'][i] ==0:
        data8_2.append(df['예매인원'][i])
    elif df['구름조금'][i-1]==1 and df['구름조금'][i] ==1:
        data8_3.append(df['예매인원'][i])

df_data8 = pd.DataFrame([['구름조금',0,sum(data8_0)/len(data8_0)/diff,len(data8_0)],
                         ['구름조금',1,sum(data8_1)/len(data8_1)/diff,len(data8_1)],
                         ['구름조금',2,sum(data8_2)/len(data8_2)/diff,len(data8_2)],
                         ['구름조금',3,sum(data8_3)/len(data8_3)/diff,len(data8_3)]],columns = index)




data9_0 = []
data9_1 = []
data9_2 = []
data9_3 = []


for i in range(2,3653):
    if df['안개'][i-1]==0 and df['안개'][i] ==0:
        data9_0.append(df['예매인원'][i])
    elif df['안개'][i-1]==0 and df['안개'][i] ==1:
        data9_1.append(df['예매인원'][i])
    elif df['안개'][i-1]==1 and df['안개'][i] ==0:
        data9_2.append(df['예매인원'][i])
    elif df['안개'][i-1]==1 and df['안개'][i] ==1:
        data9_3.append(df['예매인원'][i])

df_data9 = pd.DataFrame([['안개',0,(sum(data9_0)/len(data9_0)-diff)/diff,len(data9_0)],
                         ['안개',1,(sum(data9_1)/len(data9_1)-diff)/diff,len(data9_1)],
                         ['안개',2,(sum(data9_2)/len(data9_2)-diff)/diff,len(data9_2)],
                         ['안개',3,(sum(data9_3)/len(data9_3)-diff)/diff,len(data9_3)]],columns = index)




data10_0 = []
data10_1 = []
data10_2 = []
data10_3 = []


for i in range(2,3653):
    if df['소낙눈'][i-1]==0 and df['소낙눈'][i] ==0:
        data10_0.append(df['예매인원'][i])
    elif df['소낙눈'][i-1]==0 and df['소낙눈'][i] ==1:
        data10_1.append(df['예매인원'][i])
    elif df['소낙눈'][i-1]==1 and df['소낙눈'][i] ==0:
        data10_2.append(df['예매인원'][i])
    elif df['소낙눈'][i-1]==1 and df['소낙눈'][i] ==1:
        data10_3.append(df['예매인원'][i])

df_data10 = pd.DataFrame([['소낙눈',0,(sum(data10_0)/len(data10_0)-diff)/diff,len(data10_0)],
                         ['소낙눈',1,(sum(data10_1)/len(data10_1)-diff)/diff,len(data10_1)],
                         ['소낙눈',2,(sum(data10_2)/len(data10_2)-diff)/diff,len(data10_2)],
                         ['소낙눈',3,(sum(data10_3)/len(data10_3)-diff)/diff,len(data10_3)]],columns = index)




data11_0 = []
data11_1 = []
data11_2 = []
data11_3 = []


for i in range(2,3653):
    if df['소나기'][i-1]==0 and df['소나기'][i] ==0:
        data11_0.append(df['예매인원'][i])
    elif df['소나기'][i-1]==0 and df['소나기'][i] ==1:
        data11_1.append(df['예매인원'][i])
    elif df['소나기'][i-1]==1 and df['소나기'][i] ==0:
        data11_2.append(df['예매인원'][i])
    elif df['소나기'][i-1]==1 and df['소나기'][i] ==1:
        data11_3.append(df['예매인원'][i])

df_data11 = pd.DataFrame([['소나기',0,(sum(data11_0)/len(data11_0)-diff)/diff,len(data11_0)],
                         ['소나기',1,(sum(data11_1)/len(data11_1)-diff)/diff,len(data11_1)],
                         ['소나기',2,(sum(data11_2)/len(data11_2)-diff)/diff,len(data11_2)],
                         ['소나기',3,(sum(data11_3)/len(data11_3)-diff)/diff,len(data11_3)]],columns = index)




data12_0 = []
data12_1 = []
data12_2 = []
data12_3 = []


for i in range(2,3653):
    if df['이슬비'][i-1]==0 and df['이슬비'][i] ==0:
        data12_0.append(df['예매인원'][i])
    elif df['이슬비'][i-1]==0 and df['이슬비'][i] ==1:
        data12_1.append(df['예매인원'][i])
    elif df['이슬비'][i-1]==1 and df['이슬비'][i] ==0:
        data12_2.append(df['예매인원'][i])
    elif df['이슬비'][i-1]==1 and df['이슬비'][i] ==1:
        data12_3.append(df['예매인원'][i])

df_data12 = pd.DataFrame([['이슬비',0,(sum(data12_0)/len(data12_0)-diff)/diff,len(data12_0)],
                         ['이슬비',1,(sum(data12_1)/len(data12_1)-diff)/diff,len(data12_1)],
                         ['이슬비',2,(sum(data12_2)/len(data12_2)-diff)/diff,len(data12_2)],
                         ['이슬비',3,(sum(data12_3)/len(data12_3)-diff)/diff,len(data12_3)]],columns = index)




data13_0 = []
data13_1 = []
data13_2 = []
data13_3 = []


for i in range(2,3653):
    if df['천둥번개'][i-1]==0 and df['천둥번개'][i] ==0:
        data13_0.append(df['예매인원'][i])
    elif df['천둥번개'][i-1]==0 and df['천둥번개'][i] ==1:
        data13_1.append(df['예매인원'][i])
    elif df['천둥번개'][i-1]==1 and df['천둥번개'][i] ==0:
        data13_2.append(df['예매인원'][i])
    elif df['천둥번개'][i-1]==1 and df['천둥번개'][i] ==1:
        data13_3.append(df['예매인원'][i])

df_data13 = pd.DataFrame([['천둥번개',0,(sum(data13_0)/len(data13_0)-diff)/diff,len(data13_0)],
                         ['천둥번개',1,(sum(data13_1)/len(data13_1)-diff)/diff,len(data13_1)],
                         ['천둥번개',2,(sum(data13_2)/len(data13_2)-diff)/diff,len(data13_2)],
                         ['천둥번개',3,(sum(data13_3)/len(data13_3)-diff)/diff,len(data13_3)]],columns = index)




data14_0 = []
data14_1 = []
data14_2 = []
data14_3 = []


for i in range(2,3653):
    if df['우박'][i-1]==0 and df['우박'][i] ==0:
        data14_0.append(df['예매인원'][i])
    elif df['우박'][i-1]==0 and df['우박'][i] ==1:
        data14_1.append(df['예매인원'][i])
    elif df['우박'][i-1]==1 and df['우박'][i] ==0:
        data14_2.append(df['예매인원'][i])
    elif df['우박'][i-1]==1 and df['우박'][i] ==1:
        data14_3.append(df['예매인원'][i])

df_data14 = pd.DataFrame([['우박',0,(sum(data14_0)/len(data14_0)-diff)/diff,len(data14_0)],
                         ['우박',1,(sum(data14_1)/len(data14_1)-diff)/diff,len(data14_1)],
                         ['우박',2,(sum(data14_2)/len(data14_2)-diff)/diff,len(data14_2)],
                         ['우박',3,np.nan,len(data14_3)]],columns = index)



In [18]:
df_data1

,기준 날씨,날씨 변화,평균 변화율,데이터 갯수
0,맑음,0,0.026286,2263
1,맑음,1,-0.026490,503
2,맑음,2,0.028538,503
3,맑음,3,-0.147758,382


### 편미분 결과 합치기

,기준 날씨,날씨 변화,평균 변화율,데이터 갯수
0,맑음,0,0.026286,2263
1,맑음,1,-0.026490,503
2,맑음,2,0.028538,503
3,맑음,3,-0.147758,382
0,비,0,-0.062185,2222
1,비,1,0.394051,511
2,비,2,-0.002145,511
3,비,3,-0.142548,407
0,눈,0,-0.032883,3387
1,눈,1,2.252765,112


### 날씨 변화별 평균 변화율의 평균  즉 어느 날씨가 예매율에 가장 많이 영향을 끼치는지 확인이 가능함

In [281]:
df_diff = pd.concat([df_data1,df_data2,df_data3,df_data4,df_data5,df_data6,df_data7,df_data8,df_data9,df_data10,df_data11,df_data12,df_data13,df_data14])

df_diff_abs = df_diff.copy()
df_diff_abs['평균 변화율'] = df_diff_abs['평균 변화율'].abs()
df_diff_abs

df_diff_abs.groupby('기준 날씨').mean().sort_values(by='평균 변화율')

,날씨 변화,평균 변화율,데이터 갯수
기준 날씨,,,
맑음,1.5,0.057268,912.75
비,1.5,0.150232,912.75
이슬비,1.5,0.190618,912.75
안개,1.5,0.236122,912.75
천둥번개,1.5,0.335915,912.75
진눈개비,1.5,0.735869,912.75
소나기,1.5,0.770782,912.75
우박,1.5,0.908958,912.75
구름조금,1.5,0.975904,912.75


### 테스트 케이스 결과 불러오기

In [282]:
df_diff_test = pd.DataFrame(pd.read_csv("./test_result.csv"))

df_diff_test

,Unnamed: 0,기준 날씨,날씨 변화,평균 변화율,데이터 갯수
0,0,맑음,0,-0.092618,220
1,1,맑음,1,0.016530,52
2,2,맑음,2,0.714483,51
3,3,맑음,3,-0.277564,40
4,0,비,0,-0.074488,215
5,1,비,1,-0.997865,45
6,2,비,2,2.403103,45
7,3,비,3,-0.713812,58
8,0,눈,0,0.062452,329
9,1,눈,1,-3.024235,14


In [283]:
result = [ data for data in df_diff_test['평균 변화율']]
result

result2 = [ data for data in df_diff_test['데이터 갯수']]
result2

df_diff['테스트 케이스 평균 변화율'] = result
df_diff['테스트 케이스 데이터 갯수'] = result2
df_diff

,기준 날씨,날씨 변화,평균 변화율,데이터 갯수,테스트 케이스 평균 변화율,테스트 케이스 데이터 갯수
0,맑음,0,0.026286,2263,-0.092618,220
1,맑음,1,-0.026490,503,0.016530,52
2,맑음,2,0.028538,503,0.714483,51
3,맑음,3,-0.147758,382,-0.277564,40
0,비,0,-0.062185,2222,-0.074488,215
1,비,1,0.394051,511,-0.997865,45
2,비,2,-0.002145,511,2.403103,45
3,비,3,-0.142548,407,-0.713812,58
0,눈,0,-0.032883,3387,0.062452,329
1,눈,1,2.252765,112,-3.024235,14


In [284]:
df_diff["오차율"] = (df_diff['평균 변화율'] - df_diff['테스트 케이스 평균 변화율']*100)/df_diff['평균 변화율']
df_diff

,기준 날씨,날씨 변화,평균 변화율,데이터 갯수,테스트 케이스 평균 변화율,테스트 케이스 데이터 갯수,오차율
0,맑음,0,0.026286,2263,-0.092618,220,353.342322
1,맑음,1,-0.026490,503,0.016530,52,63.399932
2,맑음,2,0.028538,503,0.714483,51,-2502.653451
3,맑음,3,-0.147758,382,-0.277564,40,-186.849893
0,비,0,-0.062185,2222,-0.074488,215,-118.784469
1,비,1,0.394051,511,-0.997865,45,254.232480
2,비,2,-0.002145,511,2.403103,45,112016.409496
3,비,3,-0.142548,407,-0.713812,58,-499.752837
0,눈,0,-0.032883,3387,0.062452,329,190.920353
1,눈,1,2.252765,112,-3.024235,14,135.245452


In [285]:
df_diff['오차율'].abs().mean()

3566.7965244511183